In [5]:
import numpy as np
import torch
import torch.utils.data as data
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim

In [8]:
data = np.load("permuted_mnist.npz")
permuted_x_train = data["train_images"]
y_train = data["train_labels"]
permuted_x_test = data["test_images"]
y_test = data["test_labels"]

print(data.files)
print(permuted_x_test.shape)

permuted_x_train, permuted_x_val, y_train, y_val = train_test_split(
    permuted_x_train, y_train, test_size=0.1, random_state=42
)

# Convert data to PyTorch tensors
permuted_x_train = torch.from_numpy(permuted_x_train).float()
y_train = torch.from_numpy(y_train).long()
permuted_x_val = torch.from_numpy(permuted_x_val).float()
y_val = torch.from_numpy(y_val).long()
permuted_x_test = torch.from_numpy(permuted_x_test).float()
y_test = torch.from_numpy(y_test).long()

['train_images', 'train_labels', 'test_images', 'test_labels']
(10000, 28, 28)


In [3]:
class PermutedMNISTCNN(nn.Module):
    def __init__(self, num_permutations, kernel_size=3, pool_size=2, stride=2, dropout_rate=0.5):
        super(PermutedMNISTCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=kernel_size, padding=int((kernel_size - 1) / 2))
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=pool_size, stride=stride)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=int((kernel_size - 1) / 2))
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=pool_size, stride=stride)
        var1 = int((28 - pool_size) / stride) + 1
        self.dim = int((var1 - pool_size) / stride) + 1
        self.fc1 = nn.Linear(64 * self.dim * self.dim, 128)
        self.relu3 = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, num_permutations)  # Output should be the number of permutations

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 64 * self.dim * self.dim)
        x = self.relu3(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [12]:
num_permutations = 10  # Number of output classes
model = PermutedMNISTCNN(num_permutations)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Hyperparameters
num_epochs = 10
batch_size = 64

# Create DataLoaders
train_dataset = TensorDataset(permuted_x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(permuted_x_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_dataset = TensorDataset(permuted_x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Training loop
for epoch in range(num_epochs):
    model.train()
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.unsqueeze(1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Training Accuracy: {accuracy:.2f}%")
    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs.unsqueeze(1))
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Accuracy: {accuracy:.2f}%")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs.unsqueeze(1))
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Testing Accuracy: {accuracy:.2f}%")

print("Training completed.")

Epoch 1/10, Training Accuracy: 81.26%
Epoch 1/10, Validation Accuracy: 92.10%
Epoch 1/10, Testing Accuracy: 92.23%
Epoch 2/10, Training Accuracy: 89.62%
Epoch 2/10, Validation Accuracy: 93.25%
Epoch 2/10, Testing Accuracy: 93.21%
Epoch 3/10, Training Accuracy: 91.24%
Epoch 3/10, Validation Accuracy: 93.75%
Epoch 3/10, Testing Accuracy: 93.58%
Epoch 4/10, Training Accuracy: 92.28%
Epoch 4/10, Validation Accuracy: 94.45%
Epoch 4/10, Testing Accuracy: 94.44%
Epoch 5/10, Training Accuracy: 92.74%
Epoch 5/10, Validation Accuracy: 94.78%
Epoch 5/10, Testing Accuracy: 94.86%
Epoch 6/10, Training Accuracy: 93.43%
Epoch 6/10, Validation Accuracy: 95.10%
Epoch 6/10, Testing Accuracy: 95.25%
Epoch 7/10, Training Accuracy: 93.92%
Epoch 7/10, Validation Accuracy: 95.53%
Epoch 7/10, Testing Accuracy: 95.32%
Epoch 8/10, Training Accuracy: 94.33%
Epoch 8/10, Validation Accuracy: 95.70%
Epoch 8/10, Testing Accuracy: 95.53%
Epoch 9/10, Training Accuracy: 94.61%
Epoch 9/10, Validation Accuracy: 95.18%
Ep